In [4]:
# 1. Check if MA is trending
# 2. Bear candle to be followed by Bull Candle
# 3. Either Bear or Bull should pierce the MA

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
import math
import matplotlib.pyplot as plt
import talib as ta


In [ ]:
def backtest_44MA_Touchdown(df, ticker, investment):
    close = df['Close']
    open = df['Open']
    low = df['Low']
    high = df['High']
    MA44 = df['MA44']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount=0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0
    targetPrice=0
    stopLoss=0
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and (open[i-1]>close[i-1] and open[i]<close[i]) and (low[i-1]<MA44[i] and high[i]>MA44[i]):
            share = math.floor(equity / close[i] )
            equity -= share * close[i]
            buyDate = date[i]
            entry.append((i, close[i], buyDate))
            in_position = True
            targetPrice = close[i] * 1.2
            stopLoss = close[i] * 0.95
            tradeCount += 1
            
            #print(f'Buy {share} shares at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and (not is_uptrend[i] or close[i] > targetPrice or close[i] < stopLoss):
            equity += share * close[i] - commission
            sellDate = date[i]
            exit.append((i, close[i], sellDate, equity))
            in_position = False
            
            #print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'{ticker} :\t Earning from investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, tradeCount

In [65]:
def isTrending(MA44):
    #print(MA44)
    result = all(i < j for i, j in zip(MA44, MA44[1:]))
    #print ("Is list strictly increasing ? : " + str(res))
    return result;

In [89]:
def wasFalling(MA44,close):
    test = MA44 < close
    #print (test.all())
    return test.all();

In [97]:
def backtest_44MA_Touchdown(df, ticker, investment):
    close = df['Close']
    open = df['Open']
    low = df['Low']
    high = df['High']
    MA44 = df['MA44']
    date = df['Date']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount=0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0
    targetPrice=0
    stopLoss=0
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and (open[i-2]>close[i-2] and open[i-1]<close[i-1]) and (low[i-2]<MA44[i-1] and high[i-1]>MA44[i-1]) \
        and isTrending(MA44[i-6:i-1]) and wasFalling(MA44[i-6:i-1], close[i-6:i-1]) and high[i]>high[i-1]:
            share = math.floor(equity / high[i-1] )
            equity -= share * high[i-1]
            buyDate = date[i]
            entry.append((i, high[i-1], buyDate))
            in_position = True
            targetPrice = close[i] * 1.2
            stopLoss = low[i-2]
            tradeCount += 1
            #print(f'Buy {share} shares at {round(high[i-1],2)} on {df.index[i].strftime("%Y/%m/%d")}')
            
        
        elif in_position and ( close[i] > targetPrice or close[i] < stopLoss):
            equity += share * close[i] - commission
            sellDate = date[i]
            exit.append((i, close[i], sellDate, equity))
            in_position = False
    
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'{ticker} :\t Earning from investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, tradeCount
    

In [108]:
def NSEData(tickerList, startDate, endDate):
    #Fetch NSE Ticker Data
    result=[] 
    for ticker in tickerList: 
        data = []
        symbol = ticker + ".NS" 
        df = yf.download(symbol, start=startDate) 
        data.append(symbol)
        data.append(df)
        result.append(data)

    return result    

In [107]:
# Execute for single stock

ticker = 'SBICARD.NS' 
df = yf.download(ticker, start='2014-01-01') 
df['MA44'] = ta.SMA(df['Close'], timeperiod=44)
df['Date'] = df.index
df.dropna()

entry, exit, roi, tradeCount=backtest_44MA_Touchdown(df,ticker, 10000)
entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
result['Symbol'] = ticker
result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)

finalResults = result[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','Equity']]
finalResults.to_excel('Trend_MA_TouchDown.xlsx')


[*********************100%***********************]  1 of 1 completed
SBICARD.NS :	 Earning from investing 10000 is $-718.8 	 (ROI = -7.19%) 	 Trades=1


In [109]:
#Get all Trades for given input
# 1. Download Data

startDate='2015-01-01'
endDate='2019-12-31'

#BlueChip Trend from TV is giving more returns
tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']
nseDF = NSEData(tickerList,startDate,endDate)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [110]:
# 2. Generate Calls

ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []

for data in nseDF:
    df=data[1]
    ticker=data[0]
    df['MA44'] = ta.SMA(df['Close'], timeperiod=44)
    df['Date'] = df.index
    df.dropna()
    entry, exit, roi, tradeCount=backtest_44MA_Touchdown(df,ticker, 10000)
    ROI.append(roi)
    investment.append(10000)
    totalTrades.append(tradeCount)
    tickerList.append(ticker)
    entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
    exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
    result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
    result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
    result['Symbol'] = ticker
    result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)
    finalResults.append(result)
    
resultDF = pd.DataFrame(
    {'Ticker': tickerList,
     'Investment': investment,
     'FinalReturns': ROI,
     'TotalTrades': totalTrades
    })

finalResults = pd.concat(finalResults)
finalResults
finalResults = finalResults[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','Equity']]
finalResults.to_excel('Trend_MA_TouchDown_Trades.xlsx')
resultDF.to_excel('Trend_MA_TouchDown_Final_Result.xlsx')

IOB.NS :	 Earning from investing 10000 is $-488.9 	 (ROI = -4.89%) 	 Trades=1
NHPC.NS :	 Earning from investing 10000 is $-688.6 	 (ROI = -6.89%) 	 Trades=2
UNIONBANK.NS :	 Earning from investing 10000 is $-229.9 	 (ROI = -2.3%) 	 Trades=4
IDBI.NS :	 Earning from investing 10000 is $265.5 	 (ROI = 2.65%) 	 Trades=5
PNB.NS :	 Earning from investing 10000 is $2261.85 	 (ROI = 22.62%) 	 Trades=2
IDFCFIRSTB.NS :	 Earning from investing 10000 is $-1431.35 	 (ROI = -14.31%) 	 Trades=6
BHEL.NS :	 Earning from investing 10000 is $-3532.35 	 (ROI = -35.32%) 	 Trades=6
BANKINDIA.NS :	 Earning from investing 10000 is $1986.7 	 (ROI = 19.87%) 	 Trades=1
BANKBARODA.NS :	 Earning from investing 10000 is $214.15 	 (ROI = 2.14%) 	 Trades=5
L&TFH.NS :	 Earning from investing 10000 is $-1814.9 	 (ROI = -18.15%) 	 Trades=3
ADANIPOWER.NS :	 Earning from investing 10000 is $1646.95 	 (ROI = 16.47%) 	 Trades=3
TATAPOWER.NS :	 Earning from investing 10000 is $2133.55 	 (ROI = 21.34%) 	 Trades=4
NTPC.NS :	 Ea

HDFC.NS :	 Earning from investing 10000 is $918.55 	 (ROI = 9.19%) 	 Trades=3
EICHERMOT.NS :	 Earning from investing 10000 is $7070.21 	 (ROI = 70.7%) 	 Trades=7
ASIANPAINT.NS :	 Earning from investing 10000 is $8330.25 	 (ROI = 83.3%) 	 Trades=6
